## Getting Author Information Real-Time

When we deploy the web app, we expect to take in the url to a specific reddit post. Then we will need to retrieve necessary information to make a prediction. I noticed that the past history of the author seems to be a very strong predictor, so I want to incorporate that into our web app. The following code chunks will help us retrieve that.

In [57]:
import requests
import datetime as dt
import json
import time
import praw


reddit = praw.Reddit(
    client_id="kxbUr-4PyE7DlQ",
    client_secret="Q5rIAPS9IHZ1QgOIkHNY09Y9VMxDsA",
    password="AACAXZDE",
    user_agent="testscript by u/kc_the_scraper",
    username="kc_the_scraper",
)

### Functions needed

In [65]:
def getPushShiftDataAuthor(after,before, sub, author):
    ids = []
    while after < before:
        params = '&after='+str(int(after))+'&before='+str(int(before))+'&subreddit='+str(sub)+'&author='+str(author)
        url = 'https://api.pushshift.io/reddit/search/submission/?size=100' + params
        r = requests.get(url)
        data = json.loads(r.text)['data']
       
        ids += list(map(lambda x: x['id'], data))
        after = data[-1]['created_utc'] + 1
        if len(data) < 100:
            break 
    return ids


def getPostHistory(ids, upperLimit = 50):

    if not ids:
        return 0, 1
    
    totalScore = 0
    
    limit = min(len(ids), upperLimit)
    
    ids = ids[:limit]
    
    for ID in ids:
        Sub = reddit.submission(id = ID)
        totalScore += Sub.score
    
    return limit, totalScore/limit

def getAuthorFeatures(after, submissionTime, sub, author):
    ids = getPushShiftDataAuthor(after, submissionTime - 1, sub, author)
    
    return getPostHistory(ids)




### End-to-end example.

In [64]:

url = 'https://www.reddit.com/r/wallstreetbets/comments/lrxpql/melvin_tomorrow_be_like/'
Sub = reddit.submission(url = url)

start = dt.datetime(2021,1,1).timestamp()
subreddit = Sub.subreddit
author = Sub.author
end = Sub.created_utc - 1

num, avg = getAuthorFeatures(start, end, subreddit, author)

print(f"User {author} has {num} posts in r/{subreddit}, with an average upvote of {avg}.")


User raccoon-city-crypto has 5 posts in r/wallstreetbets, with an average upvote of 1.0.
